# SfM

In [ ]:
import os

src_path = os.getcwd()
print(src_path)

## Common Configs

In [ ]:
datasets_path = '../data/datasets'
datasets_name = 'milkbox_dataset'

## Extract Features

In [ ]:
import extract_features

In [ ]:
superpoint_conf = {
    'output': 'feats-spp',
    'model': {
        'path': 'thirdparty/SuperPointPretrainedNetwork/superpoint_v1.pth',
        'name': 'spp_det'
    },
    'preprocessing': {
        'grayscale': True,
        'resize_h': 512,
        'resize_w': 512
    },
    'conf': {
        'descriptor_dim': 256,
        'nms_radius': 3,
        'max_keypoints': 4096,
        'keypoints_threshold': 0.6
    }
}

image_dir_path = os.path.join(src_path, datasets_path, datasets_name, 'images')
features_out_path = os.path.join(src_path, datasets_path, datasets_name, superpoint_conf['output'] + '.h5')
print(image_dir_path, features_out_path)

In [ ]:
extract_features.main(image_dir_path, features_out_path, superpoint_conf)